In [1]:
import pandas as pd
import os
import glob

# --- CONFIGURACIÓN ---
# Carpeta donde tienes los DENGBRxx.csv
INPUT_FOLDER = r"C:\Users\Nainho\Desktop\BRA\casos_BRA"
# Donde guardarás el resumen para la web
OUTPUT_FILE = r"C:\Users\Nainho\Proton Drive\nainho1306\My files\phD\codigos\plataforma\backend\casos_brasil_resumen.csv"

def procesar_sinan():
    all_files = glob.glob(os.path.join(INPUT_FOLDER, "DENGBR*.csv"))
    print(f"Encontrados {len(all_files)} archivos de Brasil.")

    resumen_total = []

    for f in all_files:
        print(f"Procesando {os.path.basename(f)}...")
        try:
            # Leemos solo columnas necesarias para ahorrar RAM
            # DT_SIN_PRI: Fecha primeros síntomas
            # ID_MN_RESI: Código municipio (IBGE 6 dígitos)
            df = pd.read_csv(f, usecols=["DT_SIN_PRI", "ID_MN_RESI"], encoding='latin1', sep=',') 
            
            # Limpieza de fecha
            df["date"] = pd.to_datetime(df["DT_SIN_PRI"], errors="coerce")
            df = df.dropna(subset=["date"])
            
            # Agrupar por Semana y Municipio
            # W-SUN: Semanal comenzando en domingo
            df["semana"] = df["date"].dt.to_period("W-SUN").dt.start_time
            
            # Contamos casos
            grouped = df.groupby(["semana", "ID_MN_RESI"]).size().reset_index(name="casos")
            resumen_total.append(grouped)
            
        except Exception as e:
            print(f"Error en {f}: {e}")

    # Unir todo
    print("Uniendo años...")
    df_final = pd.concat(resumen_total)
    
    # Agrupar de nuevo por si hay solapamiento de fechas entre archivos
    df_final = df_final.groupby(["semana", "ID_MN_RESI"])["casos"].sum().reset_index()
    
    # Guardar
    df_final.to_csv(OUTPUT_FILE, index=False)
    print(f"¡Listo! Archivo guardado en {OUTPUT_FILE}")

if __name__ == "__main__":
    procesar_sinan()

Encontrados 16 archivos de Brasil.
Procesando DENGBR10.csv...
Procesando DENGBR11.csv...
Procesando DENGBR12.csv...
Procesando DENGBR13.csv...
Procesando DENGBR14.csv...
Procesando DENGBR15.csv...
Procesando DENGBR16.csv...
Procesando DENGBR17.csv...
Procesando DENGBR18.csv...
Procesando DENGBR19.csv...
Procesando DENGBR20.csv...
Procesando DENGBR21.csv...
Procesando DENGBR22.csv...
Procesando DENGBR23.csv...
Procesando DENGBR24.csv...
Procesando DENGBR25.csv...
Uniendo años...
¡Listo! Archivo guardado en C:\Users\Nainho\Proton Drive\nainho1306\My files\phD\codigos\plataforma\backend\casos_brasil_resumen.csv


In [2]:
import geopandas as gpd
import os

# Ajusta la ruta a donde pusiste tu archivo
SHP_PATH = r"C:\Users\Nainho\Desktop\BRA\BR_Municipios_2024\BR_Municipios_2024.shp" 

if not os.path.exists(SHP_PATH):
    print("❌ ERROR: No encuentro el archivo .shp")
else:
    try:
        print("📂 Cargando Shapefile...")
        gdf = gpd.read_file(SHP_PATH)
        
        print("\n--- INFORMACIÓN BÁSICA ---")
        print(f"Filas (Municipios): {len(gdf)}")
        print(f"Columnas: {gdf.columns.tolist()}")
        
        print("\n--- SISTEMA DE COORDENADAS (CRS) ---")
        print(f"CRS Original: {gdf.crs}") 
        # Si dice None, o algo con 'meters', ese es el problema.
        # Leaflet NECESITA 'EPSG:4326' (Latitud/Longitud).

        print("\n--- EJEMPLO DE DATOS ---")
        print(gdf.head(3))

    except Exception as e:
        print(f"❌ Error leyendo el archivo: {e}")

📂 Cargando Shapefile...

--- INFORMACIÓN BÁSICA ---
Filas (Municipios): 5573
Columnas: ['CD_MUN', 'NM_MUN', 'CD_RGI', 'NM_RGI', 'CD_RGINT', 'NM_RGINT', 'CD_UF', 'NM_UF', 'SIGLA_UF', 'CD_REGIA', 'NM_REGIA', 'SIGLA_RG', 'CD_CONCU', 'NM_CONCU', 'AREA_KM2', 'geometry']

--- SISTEMA DE COORDENADAS (CRS) ---
CRS Original: EPSG:4674

--- EJEMPLO DE DATOS ---
    CD_MUN        NM_MUN  CD_RGI                NM_RGI CD_RGINT  \
0  2504108  Carrapateira  250015            Cajazeiras     2504   
1  1718451        Pugmil  170003  Paraíso do Tocantins     1701   
2  2104206       Fortuna  210016      Presidente Dutra     2104   

             NM_RGINT CD_UF      NM_UF SIGLA_UF CD_REGIA  NM_REGIA SIGLA_RG  \
0  Sousa - Cajazeiras    25    Paraíba       PB        2  Nordeste       NE   
1              Palmas    17  Tocantins       TO        1     Norte        N   
2    Presidente Dutra    21   Maranhão       MA        2  Nordeste       NE   

  CD_CONCU NM_CONCU  AREA_KM2  \
0     None     None    59.0

In [11]:



casos=pd.read_csv(r"C:\Users\Nainho\Proton Drive\nainho1306\My files\phD\codigos\plataforma\backend\data\casos_brasil_resumen.csv")

casos["ID_MN_RESI"]=casos["ID_MN_RESI"].astype(int)

In [19]:
gdf["ID_MN_RESI"]=gdf["CD_MUN"].str[:6].astype(int)
asss=pd.merge(casos,gdf[["ID_MN_RESI","CD_MUN"]],on=["ID_MN_RESI"],how="left")

In [20]:
asss.loc[asss["CD_MUN"].isnull()]

,semana,ID_MN_RESI,casos,CD_MUN
3489,1959-01-12,539908,1,NaN
32335,2009-12-21,539921,1,NaN
33084,2009-12-28,411836,1,NaN
34620,2010-01-04,520000,1,NaN
34767,2010-01-04,539923,1,NaN
...,...,...,...,...
1310523,2024-10-07,539920,1,NaN
1337973,2025-02-10,350000,1,NaN
1359833,2025-04-07,530160,1,NaN
1403483,2025-10-13,410000,1,NaN


In [ ]:
asss

,semana,ID_MN_RESI,casos,CD_MUN
0,1911-01-09,230440,1,2304400
1,1911-07-10,352500,1,3525003
2,1912-07-08,320420,1,3204203
3,1912-07-15,292080,1,2920809
4,1912-12-09,313130,1,3131307
...,...,...,...,...
1417322,2106-03-22,355030,2,3550308
1417323,2106-04-12,355030,3,3550308
1417324,2106-04-19,355030,1,3550308
1417325,2106-04-26,355030,1,3550308


: 